In [1]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1660 Ti


# code

In [5]:
# coding: utf-8
import pandas as pd
import time
import os
import re
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
def get_data(uid_num):
    f = r'D:\result\{name}.txt'.format(name=uid_num)
    print('openning file '+ uid_num + ".txt……")
    weibo_df = pd.read_csv(f,header=None,encoding='utf-8',sep = '\t')
    start = time.time()   
    emotion_df = emotion_caculate(weibo_df,uid_num)
    end = time.time()
    print('cost time',end-start)
    print(emotion_df)
    return emotion_df

#----------------------------------------------------------------------
def emotion_caculate(text,user_id):
    df = pd.read_excel('./sentiment/大连理工大学中文情感词汇本体NAU.xlsx')
    df = df[['词语', '词性种类', '词义数', '词义序号', '情感分类', '强度', '极性']]
    df.head()

    Happy = {}
    Good = {}
    Surprise = {}
    Anger = {}
    Sad ={}
    Fear = {}
    Disgust = {}

    #df.iterrows()功能是迭代遍历每一行
    for idx, row in df.iterrows():
        if row['情感分类'] in ['PA', 'PE']:
            Happy[row['词语']] = row['强度']
        if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
            Good[row['词语']] = row['强度']
        if row['情感分类'] in ['PC']:
            Surprise[row['词语']] = row['强度']     
        if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
            Sad[row['词语']] = row['强度']
        if row['情感分类'] in ['NI', 'NC', 'NG']:
            Fear[row['词语']] = row['强度']
        if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
            Disgust[row['词语']] = row['强度']
        if row['情感分类'] in ['NAU']:     #修改: 原NA算出来没结果
            Anger[row['词语']] = row['强度'] 

    Positive = {**Happy,**Good,**Surprise}
    Negative = {**Anger,**Sad,**Fear,**Disgust}

    wordlist = []
    for i in text[0]:
        positive = 0
        negative = 0

        anger = 0
        disgust = 0
        fear = 0
        sad = 0
        surprise = 0
        good = 0
        happy = 0

        anger_list = []
        disgust_list = []
        fear_list = []
        sad_list = []
        surprise_list = []
        good_list = []
        happy_list = []

        
        a = i.strip()
        wordlist=re.split(r"[ ]+", a)+wordlist
    wordset = set(wordlist)
    print(len(wordset))

    wordfreq = []
    for word in wordset:
        freq = wordlist.count(word)
        if word in Positive:
            positive+=freq*(Positive.get(word))
        if word in Negative:
            negative+=freq*(Negative.get(word))
        if word in Anger:
            anger+=freq*(Anger.get(word))
            anger_list.append(word)
        if word in Disgust:
            disgust+=freq*(Disgust.get(word))
            disgust_list.append(word)
        if word in Fear:
            fear+=freq*(Fear.get(word))
            fear_list.append(word)
        if word in Sad:
            sad+=freq*(Sad.get(word))
            sad_list.append(word)
        if word in Surprise:
            surprise+=freq*(Surprise.get(word))
            surprise_list.append(word)
        if word in Good:
            good+=freq*(Good.get(word))
            good_list.append(word)
        if word in Happy:
            happy+=freq*(Happy.get(word))
            happy_list.append(word)
            
    emotion_info = {
        'uid':user_id,
        'length':len(wordlist),
        'positive': positive,
        'negative': negative,
        'anger': anger,
        'disgust': disgust,
        'fear':fear,
        'good':good,
        'sadness':sad,
        'surprise':surprise,
        'happy':happy 
    }
    #print('emotion_info',pd.DataFrame(emotion_info,index=[0]))
    #indexs = ['uid','length', 'positive', 'negative', 'anger', 'disgust','fear','sadness','surprise', 'good', 'happy']
    return pd.DataFrame(emotion_info,index=[0])

#---------------------------------------情感计算---------------------------------
output = pd.DataFrame(columns = ['uid','length', 'positive', 'negative', 'anger', 'disgust','fear','sadness','surprise', 'good', 'happy'])
output.to_csv('./sentiment/emotion.csv',encoding='utf_8_sig', mode='w', index=False)
uid_path = r'D:\weibo\users.csv'
data = pd.read_csv(uid_path, usecols=["用户id"], dtype={"id": str})# 只要这一列,读取为字符串类型

for n in data["用户id"]:
    print(n)
    (get_data(str(n))).to_csv('./sentiment/emotion.csv',encoding='utf_8_sig', mode='a', header=False, index=False)
print('Done')

1034998544
openning file 1034998544.txt……
1893
cost time 4.545841932296753
          uid  length  positive  negative  anger  disgust  fear  good  \
0  1034998544    5407      1110       672      5      455    42   792   

   sadness  surprise  happy  
0      182        29    303  
1083046042
openning file 1083046042.txt……
1
cost time 4.35036039352417
          uid  length  positive  negative  anger  disgust  fear  good  \
0  1083046042    1215         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
1135939212
openning file 1135939212.txt……
4304
cost time 5.476346015930176
          uid  length  positive  negative  anger  disgust  fear  good  \
0  1135939212   16619      2721       911     12      623    96  1807   

   sadness  surprise  happy  
0      194        43    889  
1345584264
openning file 1345584264.txt……
1
cost time 5.057513475418091
          uid  length  positive  negative  anger  disgust  fear  good  \
0  1345584264

294
cost time 4.433138608932495
          uid  length  positive  negative  anger  disgust  fear  good  \
0  1954348661     506        26        40      0       32     0     9   

   sadness  surprise  happy  
0        8         0     17  
1961389907
openning file 1961389907.txt……
1
cost time 4.51093053817749
          uid  length  positive  negative  anger  disgust  fear  good  \
0  1961389907       2         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
1967998234
openning file 1967998234.txt……
1
cost time 4.141919136047363
          uid  length  positive  negative  anger  disgust  fear  good  \
0  1967998234      80         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
2098255055
openning file 2098255055.txt……
1
cost time 4.107011795043945
          uid  length  positive  negative  anger  disgust  fear  good  \
0  2098255055      68         0         0      0        0  

1
cost time 4.138916492462158
          uid  length  positive  negative  anger  disgust  fear  good  \
0  2613818215     497         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
2615228915
openning file 2615228915.txt……
1
cost time 4.671865701675415
          uid  length  positive  negative  anger  disgust  fear  good  \
0  2615228915     222         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
2654284130
openning file 2654284130.txt……
1
cost time 4.456120252609253
          uid  length  positive  negative  anger  disgust  fear  good  \
0  2654284130     225         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
2655052371
openning file 2655052371.txt……
3864
cost time 4.954102277755737
          uid  length  positive  negative  anger  disgust  fear  good  \
0  2655052371   13253      2134      1357      8     1021

243
cost time 4.39421010017395
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3023956475    7733      2590         0      0        0     0  2503   

   sadness  surprise  happy  
0        0         0     87  
3044197555
openning file 3044197555.txt……
1
cost time 4.788405179977417
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3044197555      57         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
3053819357
openning file 3053819357.txt……
1
cost time 4.1708824634552
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3053819357     305         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
3089853925
openning file 3089853925.txt……
1
cost time 4.1718385219573975
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3089853925     294         0         0      0        0   

1314
cost time 4.331645488739014
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3458633544    4306       668       273      0      220    13   404   

   sadness  surprise  happy  
0       40        14    250  
3480069400
openning file 3480069400.txt……
1
cost time 5.032041072845459
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3480069400     118         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
3484538392
openning file 3484538392.txt……
1
cost time 4.645571947097778
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3484538392     216         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
3493466205
openning file 3493466205.txt……
846
cost time 4.225697040557861
          uid  length  positive  negative  anger  disgust  fear  good  \
0  3493466205    1953       401        84      0       

...

1
cost time 4.2705793380737305
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7340227139     105         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7344300519
openning file 7344300519.txt……
1
cost time 4.272571325302124
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7344300519      44         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7345057369
openning file 7345057369.txt……
2569
cost time 4.829080581665039
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7345057369   15200      1582       959     22      696    60  1055   

   sadness  surprise  happy  
0      194        15    512  
7347287084
openning file 7347287084.txt……
1
cost time 4.215384483337402
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7347287084      28         0         0      0        

1
cost time 4.419176816940308
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7435343643     649         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7435415699
openning file 7435415699.txt……
1
cost time 4.356205701828003
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7435415699     607         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7435463889
openning file 7435463889.txt……
645
cost time 4.27380108833313
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7435463889    1400       305       135      0       68     5   214   

   sadness  surprise  happy  
0       73         5     86  
7441510268
openning file 7441510268.txt……
1
cost time 4.405254364013672
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7441510268     114         0         0      0        0  

1
cost time 4.1936047077178955
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7562156808     213         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7566403736
openning file 7566403736.txt……
1
cost time 4.139965772628784
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7566403736     129         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7577089471
openning file 7577089471.txt……
1
cost time 4.1219727993011475
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7577089471     211         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7577194813
openning file 7577194813.txt……
1
cost time 4.181794166564941
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7577194813     225         0         0      0        0 

105
cost time 4.113947868347168
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7747210324     162         4        37      0       30     0     1   

   sadness  surprise  happy  
0        7         0      3  
7749172499
openning file 7749172499.txt……
1
cost time 4.097081184387207
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7749172499     296         0         0      0        0     0     0   

   sadness  surprise  happy  
0        0         0      0  
7749692209
openning file 7749692209.txt……
1333
cost time 4.236665487289429
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7749692209    3422       523       478      0      329    42   369   

   sadness  surprise  happy  
0      110         5    163  
7751572783
openning file 7751572783.txt……
1
cost time 4.158053398132324
          uid  length  positive  negative  anger  disgust  fear  good  \
0  7751572783       4         0         0      0       

"\n#显示fear、negative数据集\nfear_content = output_df.sort_values(by='fear',ascending=False)\nprint(fear_content)\n#print(fear_content.iloc[0:5])\n\nnegative_content = output_df.sort_values(by='negative',ascending=False)\nprint(negative_content)\n#print(negative_content.iloc[0:5])\n"